In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

In [ ]:
# scikit-learn에서 데이터셋 받아오고 확인하기

california = fetch_california_housing()
df = pd.DataFrame(california.data, columns=california.feature_names)
df["Target"] = california.target
print(df.tail())

In [ ]:
# 데이터 정규화

scaler = StandardScaler()
df.iloc[:, :-1] = scaler.fit_transform(df.iloc[:, :-1])
print(df.tail())

In [ ]:
# 데이터 준비
data = torch.from_numpy(df.values).float()

x = data[:, :-1]    # 마지막 행을 제외한 행들을 x로 설정
y = data[:, -1:]    # 마지막 행만 y로 설정

print(x.shape, y.shape)

# 학습 설정
epochs = 4000       # 학습 반복 횟수
batch_size = 256    # 배치 사이즈
interval = 200      # 학습 결과를 출력할 간격

In [ ]:
# 모델 정의

model = nn.Sequential(
    nn.Linear(x.size(-1), 6),
    nn.LeakyReLU(),
    nn.Linear(6, 5),
    nn.LeakyReLU(),
    nn.Linear(5, 4),
    nn.LeakyReLU(),
    nn.Linear(4, 3),
    nn.LeakyReLU(),
    nn.Linear(3, y.size(-1))
)

print(model)
optimizer = optim.Adam(model.parameters()) # 옵티마이저 함수로 Adam 사용, 학습률 필요 X

In [ ]:
# 모델 학습
for i in range(epochs):
    # 데이터 섞기 (무작위 인덱스 생성)
    indices = torch.randperm(x.size(0))

    # 입력 데이터와 정답 레이블을 같은 순서로 섞음
    x_ = torch.index_select(x, dim=0, index=indices)
    y_ = torch.index_select(y, dim=0, index=indices)

    # 미니 배치로 분할
    x_ = x_.split(batch_size, dim=0)
    y_ = y_.split(batch_size, dim=0)

    y_hat = []                          # 예측값을 저장할 리스트
    total_loss = 0                      # 전체 loss 누적값 초기화

    # 각 미니 배치에 대해 학습
    for x_i, y_i in zip(x_, y_):
        y_hat_i = model(x_i)            # 모델 예측값 계산
        loss = F.mse_loss(y_hat_i, y_i) # MSE 손실 계산

        optimizer.zero_grad()           # 기울기 초기화
        loss.backward()                 # 역전파로 기울기 계산
        optimizer.step()                # 가중치 업데이트

        total_loss += float(loss)       # 손실 누적
        y_hat += [y_hat_i]              # 예측값 저장
        
    total_loss /= len(x_)               # epoch별 평균 손실 계산

    # 학습 진행 상황 출력
    if (i + 1) % interval == 0:
        print('Epoch %d: loss=%.4e' % (i + 1, total_loss))

# 전체 예측값과 정답 레이블을 하나로 합치기
y_hat = torch.cat(y_hat, dim=0)
y = torch.cat(y_, dim=0)

In [ ]:
# 결과 시각화

df = pd.DataFrame(torch.cat([y, y_hat], dim=1).detach().numpy(), columns=["y", "y_hat"])

sns.pairplot(df, height=5)
plt.show()